In [11]:
# 検証環境

# [こちらのサイト](https://qiita.com/anieca/items/9dfe3ef46e7b655bf3ee) を参考にした。


import sys

sys.version

'3.7.7 (default, May  6 2020, 11:45:54) [MSC v.1916 64 bit (AMD64)]'

In [12]:
import tensorflow as tf
from tensorflow.keras import layers, Model

class CBR(Model):
    def __init__(self, filters, kernel_size, strides):
        super().__init__()

        params = {
            "filters": filters,
            "kernel_size": kernel_size,
            "strides": strides,
            "padding": "same",
            "use_bias": True,
            "kernel_initializer": "he_normal",
        }

        self.layers_ = [
            layers.Conv2D(**params),
            layers.BatchNormalization(),
            layers.ReLU()
        ]


    def call(self, inputs):
        for layer in self.layers_:
            inputs = layer(inputs)
        return inputs

In [13]:
class VGG16(Model):
    def __init__(self, output_size=1000):
        super().__init__()
        self.laeyrs_=[
            CBR(64, 3, 1),
            CBR(64, 3, 1),
            layers.MaxPool2D(2, padding="same"),
            CBR(128, 3, 1), 
            CBR(128, 3, 1),
            layers.MaxPool2D(2, padding="same"),
            CBR(256, 3, 1),
            CBR(256, 3, 1),
            CBR(256, 3, 1),
            layers.MaxPool2D(2, padding="same"),
            CBR(512, 3, 1),
            CBR(512, 3, 1),
            CBR(512, 3, 1),
            layers.MaxPool2D(2, padding="same"),
            layers.Flatten(),
            layers.Dense(4096),
            layers.Dense(4096),
            layers.Dense(output_size, activation="softmax"),
        ]

    def call(self, inputs):
        for layer in self.laeyrs_:
            inputs = layer(inputs)
        return inputs

In [14]:

def load_model(input_shape=None, output_size=10):
    """モデルを読み込む
    Args:
        output_size(int):
    Returns:
        model(Object): tf.keras.Model を継承したオブジェクト
    """

    model = VGG16(output_size)
    return model

In [15]:
import os
import tensorflow as tf

from src.utils import load_dataset, get_args, get_current_time

def builtin_train(args):
    # load dataset and model
    (train_images, train_labels), (test_images, test_labels) = load_dataset(args.data)
    input_shape = train_images[: args.batch_size, :, :, :].shape
    output_size = max(train_labels) + 1
    model = load_model(input_shape=input_shape, output_size=output_size)
    model.summary()

    # set tensorboard configs
    logdir = os.path.join(args.logdir, get_current_time())
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

    # loss, optimizer, metrics, setting
    model.compile(
        optimizer = tf.keras.optimizers.Adam(),
        loss = "sparse_categorical_crossentropy",
        metrics = ["accuracy"],
    )

    # dataset config (and validation, callback config)
    fit_params = {}
    fit_params["batch_size"] = args.batch_size
    fit_params["epochs"] = args.max_epoch
    if args.step_per_epoch:
        fit_params["step_per_epoch"] = args.steps_per_epoch
    fit_params[verbose] = 1
    fit_params["callbacks"] = [tensorboard_callback]
    fit_params["validation_data"] = (test_images, test_labels)

    # start train and test
    model.fit(train_images, train_labels, **fit_params)

ModuleNotFoundError: No module named 'src'

In [19]:
os.getcwd
sys.path

['d:\\My_programing\\python\\AI\\AnyCNN_Models\\VGG16',
 'C:\\Program Files\\Microsoft VS Code',
 'c:\\Users\\rurus\\.vscode\\extensions\\ms-python.python-2020.8.109390\\pythonFiles',
 'c:\\Users\\rurus\\.vscode\\extensions\\ms-python.python-2020.8.109390\\pythonFiles\\lib\\python',
 'C:\\Users\\rurus\\Miniconda3\\python37.zip',
 'C:\\Users\\rurus\\Miniconda3\\DLLs',
 'C:\\Users\\rurus\\Miniconda3\\lib',
 'C:\\Users\\rurus\\Miniconda3',
 '',
 'C:\\Users\\rurus\\Miniconda3\\lib\\site-packages',
 'C:\\Users\\rurus\\Miniconda3\\lib\\site-packages\\win32',
 'C:\\Users\\rurus\\Miniconda3\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\rurus\\Miniconda3\\lib\\site-packages\\Pythonwin',
 'C:\\Users\\rurus\\Miniconda3\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\rurus\\.ipython']